Competition Link: https://datahack.analyticsvidhya.com/contest/black-friday/

A retail company “ABC Private Limited” wants to understand the customer purchase behaviour (specifically, purchase amount) against various products of different categories. They have shared purchase summary of various customers for selected high volume products from last month.
The data set also contains customer demographics (age, gender, marital status, city_type, stay_in_current_city), product details (product_id and product category) and Total purchase_amount from last month.

Now, they want to build a model to predict the purchase amount of customer against various products which will help them to create personalized offer for customers against different products.

Your model performance will be evaluated on the basis of your prediction of the purchase amount for the test data (test.csv), which contains similar data-points as train except for their purchase amount. Your submission needs to be in the format as shown in "SampleSubmission.csv".

We at our end, have the actual purchase amount for the test dataset, against which your predictions will be evaluated. Submissions are scored on the root mean squared error (RMSE).

In [1]:
#Necessary imports for viewing data
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
#Loading test and train data set
train_df = pd.read_csv(r'train.csv')
test_df = pd.read_csv(r'test.csv')

There are 12 columns:
User_ID, Product_ID, Gender, Age, Occupation, City_Category, Stay_In_Current_City_Years, Marital_Status, Product_Category_1, Product_Category_2, Product_Category_3, Purchase

Target column is Purchase

We need to convert the categorical variables into numbers. They are:
<br> Gender
<br> Age
<br> City_Category
<br> Stay_In_Current_City
<br> Define dictionaries to convert the categorical features into numeric

In [3]:
gender_dict = {'F':0, 'M':1}
age_dict = {'0-17':0, '18-25':1, '26-35':2, '36-45':3, '46-50':4, '51-55':5, '55+':6}
city_dict = {'A':0, 'B':1, 'C':2}
stay_dict = {'0':0, '1':1, '2':2, '3':3, '4+':4}

Use the dictionary above to convert the categorical variables into numeric

In [4]:
train_df["Gender"] = train_df["Gender"].apply(lambda x: gender_dict[x])
train_df["Age"] = train_df["Age"].apply(lambda x: age_dict[x])
train_df["City_Category"] = train_df["City_Category"].apply(lambda x: city_dict[x])
train_df["Stay_In_Current_City_Years"] = train_df["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])

Now let's create a function to give the count of the features

In [5]:
def getCountofVar(dataset_df, var_name):
    var_name_grouped = dataset_df.groupby(var_name)
    count_dict = {}
    for name, group in var_name_grouped:
        count_dict[name] = group.shape[0]
    count_list = []
    for index, row in dataset_df.iterrows():
        name = row[var_name]
        count_list.append(count_dict.get(name, 0))
    return count_list

get counts for all other features

In [6]:
train_df["User_ID_Count"] = getCountofVar(train_df,"User_ID")
train_df["Product_ID_Count"] = getCountofVar(train_df,"Product_ID")
train_df["Gender_Count"] = getCountofVar(train_df,"Gender")
train_df["Age_Count"] = getCountofVar(train_df,"Age")
train_df["Occupation_Count"] = getCountofVar(train_df,"Occupation")
train_df["City_Count"] = getCountofVar(train_df,"City_Category")
train_df["Stay_Count"] = getCountofVar(train_df,"Stay_In_Current_City_Years")
train_df["Marital_Status_Count"] = getCountofVar(train_df,"Marital_Status")
train_df["PC1_Count"] = getCountofVar(train_df,"Product_Category_1")
train_df["PC2_Count"] = getCountofVar(train_df,"Product_Category_2")
train_df["PC3_Count"] = getCountofVar(train_df,"Product_Category_3")

<br> But, before this I will impute the missing values with 0. I did not impute the missing values earlier as it will cause the counts to reflect values. Right now the count for missing values is 0.

In [7]:
train_df.fillna(0, inplace=True)

In [8]:
def getPurchaseStats(target_df,compute_df, feature_name):
    feature_grouped = compute_df.groupby(feature_name)
    min_dict = {}
    max_dict = {}
    mean_dict = {}
    twentyfive_dict = {}
    fifty_dict = {}
    seventyfive_dict = {}
    for name, group in feature_grouped:
        min_dict[name] = min(np.array(group["Purchase"]))
        max_dict[name] = max(np.array(group["Purchase"]))
        mean_dict[name] = np.mean(np.array(group["Purchase"]))
        twentyfive_dict[name] = np.percentile(np.array(group["Purchase"]),25)
        fifty_dict[name] = np.percentile(np.array(group["Purchase"]),50)
        seventyfive_dict[name] = np.percentile(np.array(group["Purchase"]),75)
    min_list = []
    max_list = []
    mean_list = []
    twentyfive_list = []
    fifty_list = []
    seventyfive_list = []
    for index, row in target_df.iterrows():
        name = row[feature_name]
        min_list.append(min_dict.get(name,0))
        max_list.append(max_dict.get(name,0))
        mean_list.append(mean_dict.get(name,0))
        twentyfive_list.append( twentyfive_dict.get(name,0))
        fifty_list.append( fifty_dict.get(name,0))
        seventyfive_list.append( seventyfive_dict.get(name,0))
    return min_list, max_list, mean_list, twentyfive_list, fifty_list, seventyfive_list

User ID and Purchase stats

In [9]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "User_ID")
train_df["User_ID_Min_Purchase"] = min_price_list
train_df["User_ID_Max_Purchase"] = max_price_list
train_df["User_ID_Mean_Purchase"] = mean_price_list
train_df["User_ID_25Per_Purchase"] = twentyfive_price_list
train_df["User_ID_50Per_Purchase"] = fifty_price_list
train_df["User_ID_75Per_Purchase"] = seventyfive_price_list

Product_ID and Purchase Stats

In [10]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "Product_ID")
train_df["Product_ID_Min_Purchase"] = min_price_list
train_df["Product_ID_Max_Purchase"] = max_price_list
train_df["Product_ID_Mean_Purchase"] = mean_price_list
train_df["Product_ID_25Per_Purchase"] = twentyfive_price_list
train_df["Product_ID_50Per_Purchase"] = fifty_price_list
train_df["Product_ID_75Per_Purchase"] = seventyfive_price_list

Gender and Purchase Stats

In [11]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df, train_df, "Gender")
train_df["Gender_Min_Purchase"] = min_price_list
train_df["Gender_Max_Purchase"] = max_price_list
train_df["Gender_Mean_Purchase"] = mean_price_list
train_df["Gender_25Per_Purchase"] = twentyfive_price_list
train_df["Gender_50Per_Purchase"] = fifty_price_list
train_df["Gender_75Per_Purchase"] = seventyfive_price_list

Age and Purchase stats

In [12]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "Age")
train_df["Age_Min_Purchase"] = min_price_list
train_df["Age_Max_Purchase"] = max_price_list
train_df["Age_Mean_Purchase"] = mean_price_list
train_df["Age_25Per_Purchase"] = twentyfive_price_list
train_df["Age_50Per_Purchase"] = fifty_price_list
train_df["Age_75Per_Purchase"] = seventyfive_price_list

Occupation and Purchase stats

In [13]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "Occupation")
train_df["Occupation_Min_Purchase"] = min_price_list
train_df["Occupation_Max_Purchase"] = max_price_list
train_df["Occupation_Mean_Purchase"] = mean_price_list
train_df["Occupation_25Per_Purchase"] = twentyfive_price_list
train_df["Occupation_50Per_Purchase"] = fifty_price_list
train_df["Occupation_75Per_Purchase"] = seventyfive_price_list

City and Purchase stats

In [14]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "City_Category")
train_df["City_Min_Purchase"] = min_price_list
train_df["City_Max_Purchase"] = max_price_list
train_df["City_Mean_Purchase"] = mean_price_list
train_df["City_25Per_Purchase"] = twentyfive_price_list
train_df["City_50Per_Purchase"] = fifty_price_list
train_df["City_75Per_Purchase"] = seventyfive_price_list

stay in current city and Purchase stats

In [15]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "Stay_In_Current_City_Years")
train_df["Stay_Min_Purchase"] = min_price_list
train_df["Stay_Max_Purchase"] = max_price_list
train_df["Stay_Mean_Purchase"] = mean_price_list
train_df["Stay_25Per_Purchase"] = twentyfive_price_list
train_df["Stay_50Per_Purchase"] = fifty_price_list
train_df["Stay_75Per_Purchase"] = seventyfive_price_list

Marital status and Purchase Stats

In [16]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "Marital_Status")
train_df["Marital_Min_Purchase"] = min_price_list
train_df["Marital_Max_Purchase"] = max_price_list
train_df["Marital_Mean_Purchase"] = mean_price_list
train_df["Marital_25Per_Purchase"] = twentyfive_price_list
train_df["Marital_50Per_Purchase"] = fifty_price_list
train_df["Marital_75Per_Purchase"] = seventyfive_price_list

PC1 and Purchase Stats

In [17]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "Product_Category_1")
train_df["PC1_Min_Purchase"] = min_price_list
train_df["PC1_Max_Purchase"] = max_price_list
train_df["PC1_Mean_Purchase"] = mean_price_list
train_df["PC1_25Per_Purchase"] = twentyfive_price_list
train_df["PC1_50Per_Purchase"] = fifty_price_list
train_df["PC1_75Per_Purchase"] = seventyfive_price_list

PC2 and Purchase

In [18]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "Product_Category_2")
train_df["PC2_Min_Purchase"] = min_price_list
train_df["PC2_Max_Purchase"] = max_price_list
train_df["PC2_Mean_Purchase"] = mean_price_list
train_df["PC2_25Per_Purchase"] = twentyfive_price_list
train_df["PC2_50Per_Purchase"] = fifty_price_list
train_df["PC2_75Per_Purchase"] = seventyfive_price_list

PC3 and Purchase

In [19]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(train_df,train_df, "Product_Category_3")
train_df["PC3_Min_Purchase"] = min_price_list
train_df["PC3_Max_Purchase"] = max_price_list
train_df["PC3_Mean_Purchase"] = mean_price_list
train_df["PC3_25Per_Purchase"] = twentyfive_price_list
train_df["PC3_50Per_Purchase"] = fifty_price_list
train_df["PC3_75Per_Purchase"] = seventyfive_price_list

In [20]:
#define X
X = train_df.drop(columns=['User_ID','Product_ID','Purchase'],axis=1)

In [21]:
#define y
y = train_df["Purchase"]

In [68]:
#baseline model
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=300, max_depth = 10, learning_rate = 0.05, objective = "reg:squarederror", min_child_weight  = 10)

In [26]:
xgb.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=10, min_child_weight=10, missing=None, n_estimators=300,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [30]:
#we will take features that have importance more than 0.002
from sklearn.feature_selection import SelectFromModel
selection = SelectFromModel(xgb, threshold=0.002, prefit=True)

In [31]:
#define X_train after selecting the top few features
train_selection = selection.transform(X_train)

In [58]:
xgb_selected_features = XGBRegressor(n_estimators=300, max_depth = 10, learning_rate = 0.05, objective = "reg:squarederror", min_child_weight  = 10)
# train model
xgb_selected_features.fit(train_selection, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=10, min_child_weight=10, missing=None, n_estimators=300,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

Now we know that we can use this model for predicting the purchase for the test data
<br> So, we will do all the steps that we did for preprocessing the training data

In [35]:
#Load the test data
test_df = pd.read_csv(r'test.csv')

In [36]:
#preprocessing of categorical features
test_df["Gender"] = test_df["Gender"].apply(lambda x: gender_dict[x])
test_df["Age"] = test_df["Age"].apply(lambda x: age_dict[x])
test_df["City_Category"] = test_df["City_Category"].apply(lambda x: city_dict[x])
test_df["Stay_In_Current_City_Years"] = test_df["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])

In [37]:
test_df["User_ID_Count"] = getCountofVar(test_df,"User_ID")
test_df["Product_ID_Count"] = getCountofVar(test_df,"Product_ID")
test_df["Gender_Count"] = getCountofVar(test_df,"Gender")
test_df["Age_Count"] = getCountofVar(test_df,"Age")
test_df["Occupation_Count"] = getCountofVar(test_df,"Occupation")
test_df["City_Count"] = getCountofVar(test_df,"City_Category")
test_df["Stay_Count"] = getCountofVar(test_df,"Stay_In_Current_City_Years")
test_df["Marital_Status_Count"] = getCountofVar(test_df,"Marital_Status")
test_df["PC1_Count"] = getCountofVar(test_df,"Product_Category_1")
test_df["PC2_Count"] = getCountofVar(test_df,"Product_Category_2")
test_df["PC3_Count"] = getCountofVar(test_df,"Product_Category_3")

In [38]:
#impute blank values
test_df.fillna(0, inplace=True)

Stats for other variables are being computed

In [39]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "User_ID")
test_df["User_ID_Min_Purchase"] = min_price_list
test_df["User_ID_Max_Purchase"] = max_price_list
test_df["User_ID_Mean_Purchase"] = mean_price_list
test_df["User_ID_25Per_Purchase"] = twentyfive_price_list
test_df["User_ID_50Per_Purchase"] = fifty_price_list
test_df["User_ID_75Per_Purchase"] = seventyfive_price_list

In [40]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "Product_ID")
test_df["Product_ID_Min_Purchase"] = min_price_list
test_df["Product_ID_Max_Purchase"] = max_price_list
test_df["Product_ID_Mean_Purchase"] = mean_price_list
test_df["Product_ID_25Per_Purchase"] = twentyfive_price_list
test_df["Product_ID_50Per_Purchase"] = fifty_price_list
test_df["Product_ID_75Per_Purchase"] = seventyfive_price_list

In [41]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df, train_df, "Gender")
test_df["Gender_Min_Purchase"] = min_price_list
test_df["Gender_Max_Purchase"] = max_price_list
test_df["Gender_Mean_Purchase"] = mean_price_list
test_df["Gender_25Per_Purchase"] = twentyfive_price_list
test_df["Gender_50Per_Purchase"] = fifty_price_list
test_df["Gender_75Per_Purchase"] = seventyfive_price_list

In [42]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "Age")
test_df["Age_Min_Purchase"] = min_price_list
test_df["Age_Max_Purchase"] = max_price_list
test_df["Age_Mean_Purchase"] = mean_price_list
test_df["Age_25Per_Purchase"] = twentyfive_price_list
test_df["Age_50Per_Purchase"] = fifty_price_list
test_df["Age_75Per_Purchase"] = seventyfive_price_list

In [43]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "Occupation")
test_df["Occupation_Min_Purchase"] = min_price_list
test_df["Occupation_Max_Purchase"] = max_price_list
test_df["Occupation_Mean_Purchase"] = mean_price_list
test_df["Occupation_25Per_Purchase"] = twentyfive_price_list
test_df["Occupation_50Per_Purchase"] = fifty_price_list
test_df["Occupation_75Per_Purchase"] = seventyfive_price_list

In [44]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "City_Category")
test_df["City_Min_Purchase"] = min_price_list
test_df["City_Max_Purchase"] = max_price_list
test_df["City_Mean_Purchase"] = mean_price_list
test_df["City_25Per_Purchase"] = twentyfive_price_list
test_df["City_50Per_Purchase"] = fifty_price_list
test_df["City_75Per_Purchase"] = seventyfive_price_list

In [45]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "Stay_In_Current_City_Years")
test_df["Stay_Min_Purchase"] = min_price_list
test_df["Stay_Max_Purchase"] = max_price_list
test_df["Stay_Mean_Purchase"] = mean_price_list
test_df["Stay_25Per_Purchase"] = twentyfive_price_list
test_df["Stay_50Per_Purchase"] = fifty_price_list
test_df["Stay_75Per_Purchase"] = seventyfive_price_list

In [46]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "Marital_Status")
test_df["Marital_Min_Purchase"] = min_price_list
test_df["Marital_Max_Purchase"] = max_price_list
test_df["Marital_Mean_Purchase"] = mean_price_list
test_df["Marital_25Per_Purchase"] = twentyfive_price_list
test_df["Marital_50Per_Purchase"] = fifty_price_list
test_df["Marital_75Per_Purchase"] = seventyfive_price_list

In [47]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "Product_Category_1")
test_df["PC1_Min_Purchase"] = min_price_list
test_df["PC1_Max_Purchase"] = max_price_list
test_df["PC1_Mean_Purchase"] = mean_price_list
test_df["PC1_25Per_Purchase"] = twentyfive_price_list
test_df["PC1_50Per_Purchase"] = fifty_price_list
test_df["PC1_75Per_Purchase"] = seventyfive_price_list

In [48]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "Product_Category_2")
test_df["PC2_Min_Purchase"] = min_price_list
test_df["PC2_Max_Purchase"] = max_price_list
test_df["PC2_Mean_Purchase"] = mean_price_list
test_df["PC2_25Per_Purchase"] = twentyfive_price_list
test_df["PC2_50Per_Purchase"] = fifty_price_list
test_df["PC2_75Per_Purchase"] = seventyfive_price_list

In [49]:
min_price_list, max_price_list, mean_price_list, twentyfive_price_list,fifty_price_list, seventyfive_price_list = getPurchaseStats(test_df,train_df, "Product_Category_3")
test_df["PC3_Min_Purchase"] = min_price_list
test_df["PC3_Max_Purchase"] = max_price_list
test_df["PC3_Mean_Purchase"] = mean_price_list
test_df["PC3_25Per_Purchase"] = twentyfive_price_list
test_df["PC3_50Per_Purchase"] = fifty_price_list
test_df["PC3_75Per_Purchase"] = seventyfive_price_list

In [50]:
#store the data in a file
test_df.to_csv(r'test_full_feature.csv',index=False)
#test_df = pd.read_csv(r'test_full_feature.csv')

In [64]:
#define test data
test_data = test_df.drop(columns=['User_ID','Product_ID'],axis=1)

In [65]:
#test data based on the features selected for the model
test_data_selection = selection.transform(test_data)

In [66]:
#predict the test data
test_df["Purchase"] = xgb_selected_features.predict(test_data_selection)

In [67]:
IDcol = ['User_ID','Product_ID']
IDcol.append("Purchase")
submission = pd.DataFrame({ x: test_df[x] for x in IDcol})
submission.to_csv(r"submission_xgb4.csv", index=False)

I uploaded the submission file and the result gave me an RMSE of 2450.52. The best result in the competition had an RMSE of 2380.41